In [1]:
import pandas as pd
import datetime
!pip install pyecharts
from pyecharts.charts import *
from pyecharts import options as opts
!pip install xlrd

  Using cached pyecharts-2.0.3-py3-none-any.whl (147 kB)
     ---------------------------------------- 75.1/75.1 kB 2.1 MB/s eta 0:00:00
  Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)


In [2]:
df1=pd.read_excel("D:/从c盘转移的桌面/github/Python Financial Engineering/Financial Engineering Project1\Business Data.xls")
df1

,sales,billing_status,billing_period,billing_amount,invoice_amount,amount_paid,amount_due,estimated_payment_date,due_date,business_collection_date,credit_term,actual_payment_date,invoice_date,customer_service
0,s101,unconfirmed,2019-05,29805.0,NaN,NaN,NaN,2019-07-31,2019-07-31,2019-08-15,60,NaN,NaN,a201
1,s102,unconfirmed,2019-05,1572.6,NaN,NaN,NaN,2019-07-31,2019-07-31,2019-08-15,60,NaN,NaN,a202
2,s103,confirmed,2019-04,487551.2,487551.2,NaN,487551.2,2019-06-30,2019-06-30,2019-07-15,60,NaN,05-16,a203
3,s104,confirmed,2019-04,378835.0,378835.0,NaN,378835.0,2019-07-31,2019-07-31,2019-08-15,90,NaN,05-08,a204
4,s105,confirmed,2019-04,326866.0,326866.0,NaN,326866.0,2019-07-31,2019-07-31,2019-08-15,90,NaN,05-10,a205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5252,s5353,confirmed,2017-07,22800.0,NaN,NaN,NaN,2017-07-31,2017-08-31,2017-09-15,30,NaN,NaN,a5453
5253,s5354,confirmed,2017-07,6483.0,NaN,NaN,NaN,2017-09-30,2017-09-30,2017-10-15,60,NaN,NaN,a5454
5254,s5355,write-off,2017-06,418795.0,418795.0,418795.0,0.0,2017-08-14,2017-08-31,2017-09-15,60,2017-08-31,11-02,a5455
5255,s5356,write-off,2017-06,86337.0,86337.0,86337.0,0.0,2017-08-31,2017-08-31,2017-09-15,60,2017-08-22,11-02,a5456


In [3]:
df2=df1.copy()

In [4]:
today_time=pd.to_datetime(df2['actual_payment_date'].fillna('0').max())

In [5]:
df2['amount_paid'] = df2.amount_paid.fillna(0)
df2['invoice_amount'] = df2.invoice_amount.fillna(0)
df2['amount_due'] = df2.amount_due.fillna(0)

In [6]:
df2['billing_period'] = pd.to_datetime(df2.billing_period)
df2['due_date'] = pd.to_datetime(df2.due_date)
df2['actual_payment_date'] = pd.to_datetime(df2.actual_payment_date).fillna(today_time)

In [7]:
df2['wether_due']=df2.apply(lambda x:0 if x.due_date==today_time else 1,axis=1)
df2['wether_due_90']=(today_time-df2.due_date).map(lambda x: 1 if x.days>=90 else 0)
df2['amount_due2']=df2['billing_amount']-df2['amount_paid']
df2['days_past_due']=df2.apply(lambda x: (x.actual_payment_date-x.due_date).days if x.amount_due2==0 else (today_time-x.due_date).days, axis=1)


In [8]:
df2['current_overdue_days']=df2.apply(lambda x: (x.days_past_due) if x.amount_due2>0 else 0, axis=1)


In [9]:
df3=df2.copy()
df3['billing_quarter']=df3['billing_period'].map(lambda x: x.to_period('Q'))
df3[(df3['billing_quarter']>='2017Q3') & (df3['billing_quarter']<='2018Q4')]

,sales,billing_status,billing_period,billing_amount,invoice_amount,amount_paid,amount_due,estimated_payment_date,due_date,business_collection_date,credit_term,actual_payment_date,invoice_date,customer_service,wether_due,wether_due_90,amount_due2,days_past_due,current_overdue_days,billing_quarter
1398,s1499,confirmed,2018-12-01,158695.0,158695.0,0.0,158695.0,2019-05-16,2019-02-28,2019-03-15,60,2019-05-17,03-21,a1599,1,0,158695.0,78,78,2018Q4
1399,s1500,confirmed,2018-12-01,132310.0,132310.0,0.0,132310.0,2019-04-05,2019-03-31,2019-04-15,90,2019-05-17,01-17,a1600,1,0,132310.0,47,47,2018Q4
1400,s1501,confirmed,2018-12-01,103570.0,103570.0,0.0,103570.0,2019-05-17,2019-02-28,2019-03-15,60,2019-05-17,01-25,a1601,1,0,103570.0,78,78,2018Q4
1401,s1502,confirmed,2018-12-01,55333.0,55333.0,0.0,55333.0,2019-05-10,2019-03-31,2019-04-15,90,2019-05-17,03-01,a1602,1,0,55333.0,47,47,2018Q4
1402,s1503,part write-off,2018-12-01,42710.0,42710.0,0.0,42710.0,2019-05-16,2019-02-28,2019-03-15,60,2019-04-09,01-30,a1603,1,0,42710.0,78,78,2018Q4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5249,s5350,write-off,2017-07-01,12450.0,12450.0,12450.0,0.0,2017-09-30,2017-09-30,2017-10-15,60,2017-10-11,11-07,a5450,1,1,0.0,11,0,2017Q3
5250,s5351,confirmed,2017-07-01,1900.0,0.0,0.0,0.0,2017-10-31,2017-10-31,2017-11-15,90,2019-05-17,NaN,a5451,1,1,1900.0,563,563,2017Q3
5251,s5352,confirmed,2017-07-01,11500.0,0.0,0.0,0.0,2017-09-30,2017-09-30,2017-10-15,60,2019-05-17,NaN,a5452,1,1,11500.0,594,594,2017Q3
5252,s5353,confirmed,2017-07-01,22800.0,0.0,0.0,0.0,2017-07-31,2017-08-31,2017-09-15,30,2019-05-17,NaN,a5453,1,1,22800.0,624,624,2017Q3


In [10]:
fn1=df3.groupby('billing_quarter')[['billing_amount']].sum()
fn1.columns=['billing_amount']
fn1

,billing_amount
billing_quarter,
2017Q2,539682.00
2017Q3,8247952.62
2017Q4,11643604.99
2018Q1,17149674.79
2018Q2,31097661.29
2018Q3,38292071.12
2018Q4,51963089.64
2019Q1,40197676.24
2019Q2,14998879.43


In [11]:
fn2=df3[df3['wether_due_90']==1].groupby('billing_quarter')[['billing_amount']].sum()
fn2.columns=['amount_due']
fn2

,amount_due
billing_quarter,
2017Q2,539682.00
2017Q3,8247952.62
2017Q4,11643604.99
2018Q1,17149674.79
2018Q2,31097661.29
2018Q3,38292071.12
2018Q4,28265677.59


In [12]:
fn3 = df3[(df3.wether_due_90==1)].groupby('billing_quarter')[['amount_due2']].sum()
fn3.columns = ['current_due_90_amount']
fn3

,current_due_90_amount
billing_quarter,
2017Q2,34550.0
2017Q3,63883.0
2017Q4,57380.0
2018Q1,64283.0
2018Q2,106930.0
2018Q3,412920.1
2018Q4,304183.0


In [13]:
final1=pd.concat([fn1,fn2,fn3],axis=1)
final1

,billing_amount,amount_due,current_due_90_amount
billing_quarter,,,
2017Q2,539682.00,539682.00,34550.0
2017Q3,8247952.62,8247952.62,63883.0
2017Q4,11643604.99,11643604.99,57380.0
2018Q1,17149674.79,17149674.79,64283.0
2018Q2,31097661.29,31097661.29,106930.0
2018Q3,38292071.12,38292071.12,412920.1
2018Q4,51963089.64,28265677.59,304183.0
2019Q1,40197676.24,NaN,NaN
2019Q2,14998879.43,NaN,NaN


In [14]:
final1['90_default_rate'] = round(final1['current_due_90_amount']/final1['amount_due'],3)

In [15]:
bar =(
    Bar()
    .add_xaxis(list(final1.index.values.astype(str)))
    .add_yaxis(
        "billing_amount",
        list(final1.billing_amount),
        yaxis_index = 0,
        color="#5793f3",
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="90_net_default_rate"),
    )
    .extend_axis(
        yaxis=opts.AxisOpts(
            name='90_net_default_rate',
            type_='value',
            min_= 0,
            max_= 0.014,
            position = 'right',
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color="#d14a61")
            ),
            axislabel_opts=opts.LabelOpts(formatter="{value}"),
        )
    )
)
bar.render_notebook() 

In [16]:
df4=df3[(df3.credit_term==60)&(df3.wether_due==1)]
fn1=df4.groupby('billing_quarter')[['billing_amount']].sum()
fn1.columns=['60_credit_term_billing_amount']
df4=df3[(df3.credit_term==60)&(df3.wether_due==1)&(df3.days_past_due>0)]
fn2=df4.groupby('billing_quarter')[['amount_due2']].sum()
fn2.columns=['60_credit_term_amount_in_collection']
df4=df3[(df3.credit_term==90)&(df3.wether_due==1)]
fn3=df4.groupby('billing_quarter')[['billing_amount']].sum()
fn3.columns=['90_credit_term_billing_amount']
df4=df3[(df3.credit_term==90)&(df3.wether_due==1)&(df3.days_past_due>0)]
fn4=df4.groupby('billing_quarter')[['amount_due2']].sum()
fn4.columns=['90_credit_term_amount_in_collection']
final2=pd.concat([fn1,fn2,fn3,fn4],axis=1)

In [17]:
final2['60_credit_term_collection_rate'] = round(final2['60_credit_term_amount_in_collection']/final2['60_credit_term_billing_amount'],3)
final2['90_credit_term_collection_rate'] = round(final2['90_credit_term_amount_in_collection']/final2['90_credit_term_billing_amount'],3)

In [18]:
line = (
    Line()
    .add_xaxis(list(final1.index.values.astype(str)))
    .add_yaxis(
        "60_credit_term_collection_rate",
        list(final2['60_credit_term_collection_rate']),
        yaxis_index=0,
        color="#675bba",
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
    title_opts=opts.TitleOpts(title="different_credit_term_collection_rate"),
    )
    .add_xaxis(list(final1.index.values.astype(str)))
    .add_yaxis(
        "90_credit_term_collection_rate",
        list(final2['90_credit_term_collection_rate']),
        yaxis_index=0,
        color="#d14a61",
        label_opts=opts.LabelOpts(is_show=False),
    )
)
line.render_notebook()

In [19]:
df6=df3[(df3.amount_due2==0)&(df3.wether_due==1)].copy()
bins = [-999,0,5,10,15,20,30,60,90,999]
labels=['0','1-5','6-10','11-15','16-20','21-30','31-60','61-90','91+']

In [20]:
df6['days_past_due']=pd.cut(df6['days_past_due'],bins=bins,labels=labels)
df6['days_past_due']

253         0
254         0
255         0
256         0
257         0
        ...  
5247    31-60
5248        0
5249    11-15
5254        0
5255        0
Name: days_past_due, Length: 4452, dtype: category
Categories (9, object): ['0' < '1-5' < '6-10' < '11-15' ... '21-30' < '31-60' < '61-90' < '91+']

In [21]:
final3=df6.groupby('days_past_due')[['credit_term']].count()
final3.columns=['number_of_recovered_bills']
final3

,number_of_recovered_bills
days_past_due,
0,2901
1-5,404
6-10,268
11-15,253
16-20,111
21-30,201
31-60,166
61-90,60
91+,88
